In [55]:
import pandas as pd
import numpy as np
import scipy as sc

import matplotlib.pyplot as plt
from seaborn import distplot

from pandas_profiling import ProfileReport

import rfpimp

#Import preprocessing methos
from sklearn.impute import SimpleImputer

#Import metrics and model selection methods from scikit learn
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

#Import models
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier

# 1 - Load Data

In [69]:
train_data = pd.read_csv('train_auto.csv')
test_data = pd.read_csv('test_auto.csv')
train_data

,INDEX,TARGET_FLAG,TARGET_AMT,KIDSDRIV,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,...,BLUEBOOK,TIF,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CAR_AGE,URBANICITY
0,1,0,0.0,0,60.0,0,11.0,"$67,349",No,$0,...,"$14,230",11,Minivan,yes,"$4,461",2,No,3,18.0,Highly Urban/ Urban
1,2,0,0.0,0,43.0,0,11.0,"$91,449",No,"$257,252",...,"$14,940",1,Minivan,yes,$0,0,No,0,1.0,Highly Urban/ Urban
2,4,0,0.0,0,35.0,1,10.0,"$16,039",No,"$124,191",...,"$4,010",4,z_SUV,no,"$38,690",2,No,3,10.0,Highly Urban/ Urban
3,5,0,0.0,0,51.0,0,14.0,NaN,No,"$306,251",...,"$15,440",7,Minivan,yes,$0,0,No,0,6.0,Highly Urban/ Urban
4,6,0,0.0,0,50.0,0,NaN,"$114,986",No,"$243,925",...,"$18,000",1,z_SUV,no,"$19,217",2,Yes,3,17.0,Highly Urban/ Urban
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8156,10297,0,0.0,0,35.0,0,11.0,"$43,112",No,$0,...,"$27,330",10,Panel Truck,yes,$0,0,No,0,8.0,z_Highly Rural/ Rural
8157,10298,0,0.0,1,45.0,2,9.0,"$164,669",No,"$386,273",...,"$13,270",15,Minivan,no,$0,0,No,2,17.0,Highly Urban/ Urban
8158,10299,0,0.0,0,46.0,0,9.0,"$107,204",No,"$332,591",...,"$24,490",6,Panel Truck,no,$0,0,No,0,1.0,Highly Urban/ Urban
8159,10301,0,0.0,0,50.0,0,7.0,"$43,445",No,"$149,248",...,"$22,550",6,Minivan,no,$0,0,No,0,11.0,Highly Urban/ Urban


In [48]:
train_data.iloc[0]

INDEX                            1
TARGET_FLAG                      0
TARGET_AMT                       0
KIDSDRIV                         0
AGE                             60
HOMEKIDS                         0
YOJ                             11
INCOME                     $67,349
PARENT1                         No
HOME_VAL                        $0
MSTATUS                       z_No
SEX                              M
EDUCATION                      PhD
JOB                   Professional
TRAVTIME                        14
CAR_USE                    Private
BLUEBOOK                   $14,230
TIF                             11
CAR_TYPE                   Minivan
RED_CAR                        yes
OLDCLAIM                    $4,461
CLM_FREQ                         2
REVOKED                         No
MVR_PTS                          3
CAR_AGE                         18
URBANICITY     Highly Urban/ Urban
Name: 0, dtype: object

# 2 - Data Analysis

- Overview of the data

In [18]:
data = train_data

print('Data general description of training data :')
print('\n - number of rows = ', len(data.index))
print('\n - number of columns = ', len(data.columns))
print('\n - number of columns = ', len(data.columns))
print('\n - number of missing values (total) =' , data.isna().sum().sum())
print('\n - number of duplicated rows = ' , data.duplicated().sum())
print('\n - columns = ' , data.columns.values)

Data general description of training data :

 - number of rows =  8161

 - number of columns =  26

 - number of columns =  26

 - number of missing values (total) = 2405

 - number of duplicated rows =  0

 - columns =  ['INDEX' 'TARGET_FLAG' 'TARGET_AMT' 'KIDSDRIV' 'AGE' 'HOMEKIDS' 'YOJ'
 'INCOME' 'PARENT1' 'HOME_VAL' 'MSTATUS' 'SEX' 'EDUCATION' 'JOB'
 'TRAVTIME' 'CAR_USE' 'BLUEBOOK' 'TIF' 'CAR_TYPE' 'RED_CAR' 'OLDCLAIM'
 'CLM_FREQ' 'REVOKED' 'MVR_PTS' 'CAR_AGE' 'URBANICITY']


- General statistics

In [22]:
train_data.describe()

,INDEX,TARGET_FLAG,TARGET_AMT,KIDSDRIV,AGE,HOMEKIDS,YOJ,TRAVTIME,TIF,CLM_FREQ,MVR_PTS,CAR_AGE
count,8161.000000,8161.000000,8161.000000,8161.000000,8155.000000,8161.000000,7707.000000,8161.000000,8161.000000,8161.000000,8161.000000,7651.000000
mean,5151.867663,0.263816,1504.324648,0.171057,44.790313,0.721235,10.499286,33.485725,5.351305,0.798554,1.695503,8.328323
std,2978.893962,0.440728,4704.026930,0.511534,8.627589,1.116323,4.092474,15.908333,4.146635,1.158453,2.147112,5.700742
min,1.000000,0.000000,0.000000,0.000000,16.000000,0.000000,0.000000,5.000000,1.000000,0.000000,0.000000,-3.000000
25%,2559.000000,0.000000,0.000000,0.000000,39.000000,0.000000,9.000000,22.000000,1.000000,0.000000,0.000000,1.000000
50%,5133.000000,0.000000,0.000000,0.000000,45.000000,0.000000,11.000000,33.000000,4.000000,0.000000,1.000000,8.000000
75%,7745.000000,1.000000,1036.000000,0.000000,51.000000,1.000000,13.000000,44.000000,7.000000,2.000000,3.000000,12.000000
max,10302.000000,1.000000,107586.136160,4.000000,81.000000,5.000000,23.000000,142.000000,25.000000,5.000000,13.000000,28.000000


- Search for duplicates and missing values in each columns :
The following columns contain missing values --> YOJ, JOB, INCOME, HOME_VAL, CAR_AGE. No problematic duplicates are observed. Serveral features have dtypes that are neither int, float nor string, and that should be fixed.

In [77]:
data = train_data
report = pd.DataFrame(columns =['n_missing' , 'n_duplicates' , 'n_distinct_values', 'dtype'] , index =data.columns)
report['n_missing'] = data.isnull().sum()
report['n_duplicates'] = [data[feature].duplicated().sum() for feature in data.columns ] 
report['n_distinct_values'] = [data[feature].nunique() for feature in data.columns]
report['dtype'] = data.dtypes
report

,n_missing,n_duplicates,n_distinct_values,dtype
TARGET_FLAG,0,8159,2,int64
KIDSDRIV,0,8156,5,int64
AGE,0,8100,61,float64
HOMEKIDS,0,8155,6,int64
YOJ,0,8139,22,float64
INCOME,0,1548,6613,float64
PARENT1,0,8159,2,object
HOME_VAL,0,3054,5107,float64
MSTATUS,0,8159,2,object
SEX,0,8159,2,object


- Searching for a strategy to handle missing values

In [28]:
train_data.JOB

0        Professional
1       z_Blue Collar
2            Clerical
3       z_Blue Collar
4              Doctor
            ...      
8156    z_Blue Collar
8157          Manager
8158              NaN
8159       Home Maker
8160         Clerical
Name: JOB, Length: 8161, dtype: object

# 3 - Preprocessing

In [79]:
def clean_data(input_data) :
    """
    Takes a pandas dataframe as an input and return a copy of this dataframe after computing the following cleaning steps :
    1 - Drop the INDEX attribute that corresponds to a wrong indexation
    2 - Drop Target_AMT in order to avoid divulgating information about the target
    3 - Convert price features to float ( 'INCOME', 'BLUEBOOK', 'HOME_VAL', 'OLDCLAIM')
    4 - Fix dtypes
    """
    
    output = input_data.copy()
    
    #1 - Removre the wrong indexation
    output.drop('INDEX', axis = 1, inplace = True)
    
    #2 -Remove Target_AMT
    output.drop('TARGET_AMT', axis = 1, inplace = True)
    
    #3 - Converts prices to float values
    output = convert_price_to_float(output , ['INCOME', 'BLUEBOOK', 'HOME_VAL', 'OLDCLAIM'] )
    
    #4 - Replace missing values
    mean_imputer = SimpleImputer(strategy = 'mean')
    mostfrequent_imputer = SimpleImputer(strategy = 'most_frequent')
    output[['AGE' , 'YOJ' , 'INCOME' , 'HOME_VAL' , 'CAR_AGE']] = mean_imputer.fit_transform(output[['AGE' , 'YOJ' , 'INCOME' , 'HOME_VAL' , 'CAR_AGE']])
    output[['JOB']] = mostfrequent_imputer.fit_transform(output[['JOB']])
    
    #5 - Get dummies
    output = pd.get_dummies(output, drop_first = True)
    
    
    
    return output



def convert_price_to_float(dataframe , features) :
    """
    Convert the specified features of a dataframe from string prices to float numbers
    """
    def price_to_float(string_price) :
        """
        Converts a string price into its float value
        """
        #removes dollar signs and replace coma by dot to enable float conversion
        new_string = str(string_price).replace('$' , '').replace(',' , '.')
        
        return(float(new_string))
        
    for feature in features :
        dataframe[feature]= dataframe[feature].map(lambda x : price_to_float(x))
    return dataframe


out = clean_data(train_data)
out.columns

Index(['TARGET_FLAG', 'KIDSDRIV', 'AGE', 'HOMEKIDS', 'YOJ', 'INCOME',
       'HOME_VAL', 'TRAVTIME', 'BLUEBOOK', 'TIF', 'OLDCLAIM', 'CLM_FREQ',
       'MVR_PTS', 'CAR_AGE', 'PARENT1_Yes', 'MSTATUS_z_No', 'SEX_z_F',
       'EDUCATION_Bachelors', 'EDUCATION_Masters', 'EDUCATION_PhD',
       'EDUCATION_z_High School', 'JOB_Doctor', 'JOB_Home Maker', 'JOB_Lawyer',
       'JOB_Manager', 'JOB_Professional', 'JOB_Student', 'JOB_z_Blue Collar',
       'CAR_USE_Private', 'CAR_TYPE_Panel Truck', 'CAR_TYPE_Pickup',
       'CAR_TYPE_Sports Car', 'CAR_TYPE_Van', 'CAR_TYPE_z_SUV', 'RED_CAR_yes',
       'REVOKED_Yes', 'URBANICITY_z_Highly Rural/ Rural'],
      dtype='object')

In [54]:
d = out.copy()
d = pd.get_dummies(d, drop_first = True)

Index(['TARGET_FLAG', 'KIDSDRIV', 'AGE', 'HOMEKIDS', 'YOJ', 'INCOME',
       'HOME_VAL', 'TRAVTIME', 'BLUEBOOK', 'TIF', 'OLDCLAIM', 'CLM_FREQ',
       'MVR_PTS', 'CAR_AGE', 'PARENT1_Yes', 'MSTATUS_z_No', 'SEX_z_F',
       'EDUCATION_Bachelors', 'EDUCATION_Masters', 'EDUCATION_PhD',
       'EDUCATION_z_High School', 'JOB_Doctor', 'JOB_Home Maker', 'JOB_Lawyer',
       'JOB_Manager', 'JOB_Professional', 'JOB_Student', 'JOB_z_Blue Collar',
       'CAR_USE_Private', 'CAR_TYPE_Panel Truck', 'CAR_TYPE_Pickup',
       'CAR_TYPE_Sports Car', 'CAR_TYPE_Van', 'CAR_TYPE_z_SUV', 'RED_CAR_yes',
       'REVOKED_Yes', 'URBANICITY_z_Highly Rural/ Rural'],
      dtype='object')

In [75]:
output = train_data.copy()
mean_imputer = SimpleImputer(strategy = 'mean')
mostfrequent_imputer = SimpleImputer(strategy = 'most_frequent')
output[['AGE' , 'YOJ' , 'INCOME' , 'HOME_VAL' , 'CAR_AGE']] = mean_imputer.fit_transform(output[['AGE' , 'YOJ' , 'INCOME' , 'HOME_VAL' , 'CAR_AGE']])
output[['JOB']] = mostfrequent_imputer.fit_transform(output[['JOB']])

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: '$67,349'

In [74]:
out

,AGE,YOJ
0,60.0,11.0
1,43.0,11.0
2,35.0,10.0
3,51.0,14.0
4,50.0,NaN
...,...,...
8156,35.0,11.0
8157,45.0,9.0
8158,46.0,9.0
8159,50.0,7.0
